In [1]:
from phoenix_wrapper.runner import PhoenixSolver

In [ ]:
solver = PhoenixSolver( 
    # working_directory="some/path/", # Specify a working directory. Default is the current directory
    # platform="windows", # Platform to run on. This only really changes if ".exe" or ".out" is appended to the executable name. Default is "windows"
)

# We can also check if we find any executables in the directory of this package that we can run.
# The solver does this automatically when its first run, but we can also do it manually.
solver.update_executables()

# We can also trigger building the executables in the directory of this package.
# This function clones https://github.com/Schumacher-Group-UPB/PHOENIX -> master and trys to build both the CPU and GPU versions.
# solver.build()

In [ ]:
# Reset current solver settings to default
solver.reset_settings()

# Generate some new settings. We can add settings using a dictionary like this:
settings = solver.import_settings( # Import settings from a dictionary. This function also returns the settings, so we can snapshot them for later use
    parameters={
        "N": (300, 300), # Number of grid points
        "L": (100, 100), # Length of the grid
        "pulse": [ # Pulse settings
            ( 5.6856e-3, "add", 0.5, 0.5, 0, 0, "both", 1, 0, "gauss+noDivide", "time", "iexp", 10, 1, -2),
            ( 5.6856e-3, "add", 0.5, 0.5, 0, 0, "both", 1, 0, "gauss+noDivide", "time", "iexp", 40, 1, -2),
        ],
        "outEvery": 0.1, # Output every 0.1ps
        "initRandom": (1e-1, 1337), # Random seed for initial conditions
        "output": "wavefunction,scalar", # Output wavefunction and scalar data
    } 
)

# The solver also provides some helper functions to set settings, like the time:
solver.p_set_time( 
    t1=100, # Set the simulation time to 100ps
    #dt = 0.001, # Set the time step to 0.001ps. Default is None (magic time step)
)

# We can export the settings into a json file, which we can also load using .import_settings(...) instead of specifying the settings in a dictionary
# solver.export_settings( path="some/path/to/settings.json" )

# Build the new runstring. This method is required to update the runstring with the new settings. The method also returns the runstring
solver.build_runstring()

In [ ]:
# The solver class can also help building and visualizing the enevelopes using the p_add_envelope_assist() method
# This method is highly subject to change and merely a prototype 
solver.p_add_envelope_assist()

In [ ]:
# Run the solver with the settings we specified, using either fp32 or fp64 precision. We can use verbose output to create progress bars in jupyter notebooks
summary = solver.run( # Run the solver. Returns a summary of the run
    precision="fp32", # Use single precision
    output_verbose=True # Output verbose output
)

print(f"Summary: {summary}")

# The solver also provideos a minimal output function that can be used in e.g. loops
# This function only returns the return code of the solver
# solver.quick_run( precision="fp64")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# We can retrieve the current parameters from the snapshot we took earlier
Nx,Ny = settings["parameters"]["N"]

# Build a random matrix
test_matrix = np.random.rand(Nx,Ny) + 1j*np.random.rand(Nx,Ny)

# We can use the solver to save any numpy matrix to a file. The solver will automatically save the matrix in the correct format
# PHOENIX can then load this file and use it
solver.save_matrix( test_matrix, "test_matrix.txt" )

# The solver can also plot matrices in the current working directory. This can include custom matrices or the output of the solver
solver.plot_matrix(
    name="wavefunction_plus.txt", # Matrix to plot
    use_abs=True, # Use the absolute value of the matrix
    use_fft=True, # Use the FFT of the matrix. These can be combined.
    fft_shift=True, # Shift the FFT
    k_space=True, # Plot the k-space instead of the real space
    title="FFT of test_matrix", # Title of the plot
    # From here, kwargs are passed to the matplotlib imshow function
    cblabel="Amplitude", # Label of the colorbar
    cmap="turbo" # Colormap to use
)
plt.show()

# The solver provides a quick-plot function to plot the scalar output of the solver
solver.plot_scalar();